# [OpenFOAM Simulations in a nutshell](https://foam-school.github.io/docs/part1/openfoam-simulation-in-a-nutshell/) (Diffusive flow simulation)

### 1. Imports and initial configuration

We make sure plots are "inlined", import an improved `DataFrame` class and prepare the case directory for processing

In [ ]:
# If you don't know Python, don't worry about these lines. Just run-n-skip -em
%matplotlib inline
from matplotlib import pyplot as plt
from PyFoam.Wrappers.Pandas import PyFoamDataFrame as DataFrame
from PyFoam.IPythonHelpers import storage
store=storage()

Then, we import the `Case` class and instantiate our case object 
(The abolute path is given _relative_ to the "docker container")

In [ ]:
from PyFoam.IPythonHelpers.Case import Case
# Pass the absolute path to the case to Case class
case=Case('/root/foam/root-4.0/run/intro')

### 2. Investigating the case

Retrieve some mesh statistics (Use `case.<TAB>` to learn more) as HTML objects

In [ ]:
# First, make sure the mesh is there
# We run shell commands with 
!blockMesh > /dev/null

# Then inspect mesh size
case.size()

In [ ]:
# Length denotes how many faces are on the boundary patch
case.boundaryConditions()

You can even verify fields' dimensions here 

In [ ]:
case.dimensions()

and also the variables values at cell centers for time 0 (initial time)

In [ ]:
case.internalField()

Finally, the numerical solver's settings

In [ ]:
case.linearSolvers()
# Or you can type "case." and press <TAB> to get a list of available methods

### Preparing and running the case

In [ ]:
## For those who don't know Python
## You run things from here as if you work from the commandline;
## Just add a ! before each shell command

! foamCleanTutorials && blockMesh
! scalarTransportFoam

In [ ]:
## For Python programmers, there is a PyFoam Package to automate 
## OpenFOAM stuff

# Use PyFoam as a library to launch simulations
from PyFoam.Applications.Runner import Runner

# On commandline, you can run:
#! pyFoamRunner.py --clear --progress scalarTransportFoam -case .

# Notice that
# PyFoam application options start with --
# OpenFOAM solver options start with a single -

# Clear the case, show only the progress,
# Choose scalarTransportFoam, and provide case absolute path as its option
rn = Runner(args=["--clear","--progress","scalarTransportFoam","-case",case.path])


### Investigating the run

In [ ]:
# Get simulation Data as a Python dictionary
data = rn.getData()

In [ ]:
# Discover what info is stored
data.keys()

In [ ]:
# Has the run finished OK?
rn.OK

In [ ]:
# If failed, is it an OpenFOAM Fatal Error?
rn.fatalError

In [ ]:
# Or did the run get interrupted
rn.interrupted

In [ ]:
# Was there a run "remark" (For over-the-network simulations)
rn.remark

In [ ]:
# CPU TIME 
rn.cpuSystemTime, rn.cpuUserTime, rn.cpuTime

In [ ]:
# Newest time directory
case.sol.latestDir()

In [ ]:
data.values()

### The list of T values

What we *really* want to extract in this case is the `nonuniform list of T` at time = 40 (40th timeStep)!

In [ ]:
case.internalField(time=40.0)
# But this is HTML!! can't plot, can't do a thing with the data

In [ ]:
# An alternative way, for this case, is to directly parse 2/T file

# Import parsedFile class as sf
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile as sf

# The 2/T file representation
T = sf(case.sol.latestDir()+'/T')

# What content the file has?
print("40/T content: %s\n" % T.content.keys())

# Extract the list of values
print("T at cell center: %s" % T.content['internalField'].value())

In [ ]:
# Plot T values at cell centers
plt.plot(T.content['internalField'].value())
plt.xlabel("Cell ID")
plt.ylabel("T")